In [ ]:
import math
import os
from typing import Optional, Tuple

from PIL import Image
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    GoogleSearchTool,
    HfApiModel,
    OpenAIServerModel,
    VisitWebpageTool,
    tool,
)
from smolagents.utils import encode_image_base64, make_image_url
from utils import gemini_model_OpenAIServer

In [ ]:
model = gemini_model_OpenAIServer("gemini-2.0-flash")

In [ ]:
@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

# first attempt

In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [ ]:
# may need to run multiple times, due to rate limits and/or parsing errors.
result = agent.run(task)

In [ ]:
result

# More prompting

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [10]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## Facts survey
### 1.1. Facts given in the task
*   Current location: Gotham (40.7128° N, 74.0060° W)
*   Need to find: Batman filming locations worldwide
*   Need to calculate: Cargo plane travel time from each location to Gotham
*   Need to find: Supercar factories with similar cargo plane travel times to Batman filming locations.
*   Output format: pandas dataframe

### 1.2. Facts to look up
*   List of Batman filming locations and their coordinates: web search "Batman filming locations" (multiple 
searches might be necessary for different movies or completeness). Source: various websites detailing film 
locations.
*   Coordinates (latitude, longitude) of the Batman filming locations identified above: Google Maps or other 
coordinate lookup services. Source: online maps.
*   List of Supercar Factories and their coordinates: Web search "Supercar Factories". Source: various websites 
detailing car manufacturing locations.
*   Cargo plane cruising speed (default is 750 km/h, confirm if a different value should be used). Source: Could 
use a web search like "average cargo plane speed".

### 1.3. Facts to derive
*   Cargo plane travel time from each Batman filming location to Gotham. Use the `calculate_cargo_travel_time` tool
with the origin and destination coordinates.
*   Cargo plane travel time from each Supercar factory location to Gotham. Use the `calculate_cargo_travel_time` 
tool with the origin and destination coordinates.
*   A pandas dataframe containing the Batman filming locations, their coordinates, and the cargo plane travel time 
to Gotham.
*   A pandas dataframe containing the Supercar factories, their coordinates, and the cargo plane travel time to 
Gotham, filtered to the same duration as the Batman locations.
## Plan

1.  Search for Batman filming locations using the `web_search` tool with the query "Batman filming locations".
2.  Visit the relevant URLs from the search results using the `visit_webpage` tool to extract specific filming 
locations and ideally their geographic coordinates.
3. If geographic coordinates are not available on the visited pages, search each filming location name with the 
`web_search` tool to find its coordinates.
4.  Iterate through each Batman filming location.  For each location, use the `calculate_cargo_travel_time` tool to
determine the travel time to Gotham (40.7128° N, 74.0060° W).
5.  Store the Batman filming locations, their coordinates, and the calculated travel times in a list of 
dictionaries.
6.  Search for Supercar factories using the `web_search` tool with the query "Supercar Factories".
7.  Visit the relevant URLs from the search results using the `visit_webpage` tool to extract specific Supercar 
Factory locations and ideally their geographic coordinates.
8. If geographic coordinates are not available on the visited pages, search each factory location name with the 
`web_search` tool to find its coordinates.
9.  Iterate through each Supercar factory location. For each location, use the `calculate_cargo_travel_time` tool 
to determine the travel time to Gotham (40.7128° N, 74.0060° W).
10. Store the Supercar factories, their coordinates, and the calculated travel times in a list of dictionaries.
11. Filter the Supercar factories to those with travel times similar to the Batman filming locations. "Similar" 
will require defining a tolerance range (e.g., +/- 1 hour).
12. Create a pandas dataframe from the Batman filming locations data.
13. Create a pandas dataframe from the filtered Supercar factory data.
14. Return both dataframes as the final answer using the `final_answer` tool.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_search_results = web_search(query="Batman filming locations")                                             
  print(batman_search_results)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'batman_search_results = web_search(query="Batman filming locations")' due to: 
DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 403 Ratelimit

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Batman Filming Locations Data                                                                                  
  filming_locations = {                                                                                            
      "Chicago": (41.8781, -87.6298),                                                                              
      "London": (51.5074, 0.1278),                                                                                 
      "Hong Kong": (22.3193, 114.1694)                                                                             
  }                                                                                                                
                                                                                                                   
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  travel_times = {}                                                                                                
  for location, coords in filming_locations.items():                                                               
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      travel_times[location] = travel_time                                                                         
                                                                                                                   
  batman_data = []                                                                                                 
  for location, coords in filming_locations.items():                                                               
      batman_data.append({                                                                                         
          "Location": location,                                                                                    
          "Latitude": coords[0],                                                                                   
          "Longitude": coords[1],                                                                                  
          "Travel Time (hours)": travel_times[location]                                                            
      })                                                                                                           
                                                                                                                   
  batman_df = pd.DataFrame(batman_data)                                                                            
                                                                                                                   
  # Supercar Factories Data                                                                                        
  supercar_factories = {                                                                                           
      "Maranello": (44.5373, 10.8604),                                                                             
      "Sant'Agata Bolognese": (44.6667, 11.1333),                                                                  
      "Stuttgart": (48.7758, 9.1829)                                                                               
  }                                                                                                                
                                                         

Out - Final answer: {'Batman Filming Locations': '    Location  Latitude  Longitude  Travel Time (hours)\n0    
Chicago   41.8781   -87.6298                 2.68\n1     London   51.5074     0.1278                 9.19\n2  Hong 
Kong   22.3193   114.1694                20.00', 'Supercar Factories with similar travel times': {'Chicago': [], 
'London': ['Maranello', "Sant'Agata Bolognese", 'Stuttgart'], 'Hong Kong': []}}

[Step 10: Duration 3.91 seconds| Input tokens: 57,606 | Output tokens: 3,086]

{'Batman Filming Locations': '    Location  Latitude  Longitude  Travel Time (hours)\n0    Chicago   41.8781   -87.6298                 2.68\n1     London   51.5074     0.1278                 9.19\n2  Hong Kong   22.3193   114.1694                20.00', 'Supercar Factories with similar travel times': {'Chicago': [], 'London': ['Maranello', "Sant'Agata Bolognese", 'Stuttgart'], 'Hong Kong': []}}


In [11]:
detailed_report

{'Batman Filming Locations': '    Location  Latitude  Longitude  Travel Time (hours)\n0    Chicago   41.8781   -87.6298                 2.68\n1     London   51.5074     0.1278                 9.19\n2  Hong Kong   22.3193   114.1694                20.00',
 'Supercar Factories with similar travel times': {'Chicago': [],
  'London': ['Maranello', "Sant'Agata Bolognese", 'Stuttgart'],
  'Hong Kong': []}}

# Splitting the task between two agents

In [22]:
web_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [ ]:
gemini_pro_model_name = "gemini-2.0-pro-exp-02-05"  # maybe rate limited
gemini_flash_model_name = "gemini-2.0-flash"

In [27]:
# The manager agent will need to do some mental heavy lifting. So we give it the stronger model gemini-pro,
# and add a planning_interval to the mix.
def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = gemini_model_OpenAIServer(gemini_flash_model_name)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=gemini_model_OpenAIServer(gemini_flash_model_name),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

NameError: name 'gemini_flash_model_name' is not defined

In [28]:
manager_agent.visualize()

CodeAgent | gemini-2.0-pro-exp-02-05
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gemini-2.0-flash
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │                             

In [29]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-pro-exp-02-05 ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## Facts survey
### 1.1. Facts given in the task
*   Our current location: Gotham (40.7128° N, 74.0060° W).
*   Default cruising speed of cargo plane: 750 km/h (mentioned in the `calculate_cargo_travel_time` tool 
description).
*   Example of plotting the required kind of graph.

### 1.2. Facts to look up
*   Batman filming locations worldwide. Source: web_agent.
*   Coordinates (latitude and longitude) of each Batman filming location. Source: web_agent.
*   Supercar factories worldwide. Source: web_agent.
*   Coordinates (latitude and longitude) of each supercar factory. Source: web_agent.

### 1.3. Facts to derive
*   Travel time from each Batman filming location to Gotham. Source: `calculate_cargo_travel_time` tool, using the 
coordinates of Gotham and each filming location.
*   Travel time from each supercar factory to Gotham. Source: `calculate_cargo_travel_time` tool, using the 
coordinates of Gotham and each factory.
*   Supercar factories with the same cargo plane transfer time (within a reasonable tolerance) as the Batman 
filming locations. Source: Comparison of travel times calculated above. We need to define what the acceptable 
tolerance would be.
* Select at least 6 points to plot.

## Plan
1.  **Find Batman filming locations:** Task the `web_agent` to find a comprehensive list of Batman filming 
locations worldwide, including their names and, if possible, coordinates (latitude and longitude).
2.  **Get coordinates for filming locations:** If the `web_agent` did not provide coordinates, task the `web_agent`
to find the latitude and longitude for each filming location.
3.  **Calculate travel times for filming locations:** For each filming location, use the 
`calculate_cargo_travel_time` tool to calculate the travel time to Gotham (40.7128° N, 74.0060° W).
4.  **Find supercar factory locations:** Task the `web_agent` to find a list of supercar factories worldwide. It 
would help the task if we could give it some search constrains, for example by continent, if it is possible to give
such constrains.
5.  **Get coordinates for supercar factories:** Task the `web_agent` to find the latitude and longitude for each 
supercar factory.
6.  **Calculate travel times for supercar factories:** For each supercar factory, use the 
`calculate_cargo_travel_time` tool to calculate the travel time to Gotham (40.7128° N, 74.0060° W).
7. **Find matching transfer Times**: Compare travel time between filming locations, and supercar factories.
8. **Prepare data for plotting**: Collect a list of at least 6 locations. Include a mix of Batman filming locations
and supercar factories that are in a similar range of transfer times.
9.  **Create the map:** Create a scatter map using `plotly.express`, with latitude and longitude as coordinates, 
and color representing the travel time.
10. **Save and output:** Save the generated map as "saved_map.png" and provide it to the `final_answer` tool.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   
                                                                                                                   
Thought: Let's start by finding Batman filming locations.                                                          
Code:                                                                                                              
```py                                                                                                              
web_agent(task="Find a list of Batman filming locations worldwide. Please include the city, country, and if        
available, the exact location name (e.g., Wayne Tower, Gotham City, USA).")                                        
```</end_code>                                                                                                     

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  web_agent(task="Find a list of Batman filming locations worldwide. Please include the city, country, and if      
  available, the exact location name (e.g., Wayne Tower, Gotham City, USA).")                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

[Step 9: Duration 11.04 seconds| Input tokens: 81,664 | Output tokens: 2,564]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Error code: 429 - [{'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 
'RESOURCE_EXHAUSTED'}}\]

[Step 10: Duration 1.64 seconds| Input tokens: 97,559 | Output tokens: 2,678]

AgentGenerationError: Error in generating model output:
Error code: 429 - [{'error': {'code': 429, 'message': 'Resource has been exhausted (e.g. check quota).', 'status': 'RESOURCE_EXHAUSTED'}}]

In [ ]:
manager_agent.python_executor.state["fig"]